# SBERT & TF-IDF 기반 Netflix 추천 시스템

이 노트북에서는 사전에 생성된 SBERT 임베딩과 TF-IDF 벡터를 활용하여 개인 및 그룹 추천 시스템을 구현합니다.

## 주요 기능
- 백엔드 API에서 시청기록 조회
- SBERT 및 TF-IDF 모델을 활용한 유사도 계산
- 개인 추천 (Individual Recommendation)
- 그룹 추천 (Group Recommendation)
- 성능 비교 및 분석

## 1. 필요한 라이브러리 및 설정

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import joblib
import time
import warnings
warnings.filterwarnings('ignore')

# 환경변수 로드
load_dotenv()

print("라이브러리 로딩 완료")
print(f"API_URL: {os.environ.get('API_URL')}")

라이브러리 로딩 완료
API_URL: http://3.36.57.234:8080/api


## 2. 데이터 및 모델 로드

In [2]:
# 데이터 로드
print("데이터 로딩 중...")
df = pd.read_csv('preprocessed_video.csv')
print(f"전체 영화/TV 쇼 개수: {len(df)}")

# NaN 값 처리
df = df.fillna('')

# SBERT 임베딩 로드
print("\nSBERT 임베딩 로드 중...")
sbert_embeddings = np.load('sbert_embeddings.npy')
print(f"SBERT 임베딩 크기: {sbert_embeddings.shape}")

# TF-IDF 모델 및 행렬 로드
print("\nTF-IDF 모델 로드 중...")
tfidf_vectorizer = joblib.load('tfidf_vectorizer_optimized.joblib')
tfidf_matrix = np.load('tfidf_matrix_optimized.npy')
print(f"TF-IDF 행렬 크기: {tfidf_matrix.shape}")

print("\n모든 데이터 로딩 완료!")

데이터 로딩 중...
전체 영화/TV 쇼 개수: 5317

SBERT 임베딩 로드 중...
SBERT 임베딩 크기: (5317, 384)

TF-IDF 모델 로드 중...
TF-IDF 행렬 크기: (5317, 15000)

모든 데이터 로딩 완료!


In [3]:
df

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,month_name_added,year_added
0,1451,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",9,September,2021
1,1452,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,2021-09-24,2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,9,September,2021
2,1453,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021-09-24,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,9,September,2021
3,1454,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021-09-23,2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...,9,September,2021
4,1455,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,2021-09-21,1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...,9,September,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312,6763,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan",2016-03-09,2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...,3,March,2016
5313,6764,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",11,November,2019
5314,6765,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,11,November,2019
5315,6766,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",1,January,2020


## 3. API 연동 함수 정의

In [4]:
def get_user_history_ids(user_id: int):
    """사용자의 시청기록 ID 목록 조회"""
    url = f"{os.environ.get('API_URL')}/users/{user_id}"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    
    user_history_ids = [item['id'] for item in data.get('histories', [])]
    return user_history_ids

def get_group_history_ids(group_id: int):
    """그룹의 시청기록 ID 목록 조회"""
    url = f"{os.environ.get('API_URL')}/groups/{group_id}/histories"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    
    group_history_ids = [item['id'] for item in data]
    return group_history_ids

def get_video_detail(video_id: int):
    """비디오 상세 정보 조회"""
    url = f"{os.environ.get('API_URL')}/videos/{video_id}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

print("API 연동 함수 정의 완료")

API 연동 함수 정의 완료


## 4. SBERT 기반 추천 함수

In [5]:
def get_sbert_recommendations(history_ids, top_k=10):
    """SBERT 임베딩을 활용한 추천 시스템"""
    # 시청기록 매칭
    watched_mask = np.isin(df['id'].values, history_ids)
    watched_idx = np.where(watched_mask)[0]
    
    if len(watched_idx) == 0:
        return []
    
    # 사용자/그룹 프로필 벡터 생성 (평균)
    profile_vector = sbert_embeddings[watched_idx].mean(axis=0)
    
    # 모든 콘텐츠와의 유사도 계산
    similarities = cosine_similarity([profile_vector], sbert_embeddings)[0]
    
    # 시청한 콘텐츠 제외
    similarities[watched_idx] = -1
    
    # 상위 k개 추천
    top_indices = similarities.argsort()[::-1][:top_k]
    
    recommendations = []
    for idx in top_indices:
        if similarities[idx] > 0:  # 유효한 유사도만
            recommendations.append({
                'id': int(df.iloc[idx]['id']),
                'title': df.iloc[idx]['title'],
                'director': df.iloc[idx]['director'],
                'rating': df.iloc[idx]['rating'],
                'type': df.iloc[idx]['type'],
                'similarity': float(similarities[idx]),
                'genre': df.iloc[idx]['listed_in']
            })
    
    return recommendations

print("SBERT 추천 함수 정의 완료")

SBERT 추천 함수 정의 완료


## 5. TF-IDF 기반 추천 함수

In [6]:
def get_tfidf_recommendations(history_ids, top_k=10):
    """TF-IDF 벡터를 활용한 추천 시스템"""
    # 시청기록 매칭
    watched_mask = np.isin(df['id'].values, history_ids)
    watched_idx = np.where(watched_mask)[0]
    
    if len(watched_idx) == 0:
        return []
    
    # 사용자/그룹 프로필 벡터 생성 (평균)
    profile_vector = tfidf_matrix[watched_idx].mean(axis=0)
    
    # 2D 배열로 reshape (cosine_similarity 요구사항)
    if profile_vector.ndim == 1:
        profile_vector = profile_vector.reshape(1, -1)
    
    # 모든 콘텐츠와의 유사도 계산
    similarities = cosine_similarity(profile_vector, tfidf_matrix)[0]
    
    # 시청한 콘텐츠 제외
    similarities[watched_idx] = -1
    
    # 상위 k개 추천
    top_indices = similarities.argsort()[::-1][:top_k]
    
    recommendations = []
    for idx in top_indices:
        if similarities[idx] > 0:  # 유효한 유사도만
            recommendations.append({
                'id': int(df.iloc[idx]['id']),
                'title': df.iloc[idx]['title'],
                'director': df.iloc[idx]['director'],
                'rating': df.iloc[idx]['rating'],
                'type': df.iloc[idx]['type'],
                'similarity': float(similarities[idx]),
                'genre': df.iloc[idx]['listed_in']
            })
    
    return recommendations

print("TF-IDF 추천 함수 정의 완료 (차원 문제 해결)")

TF-IDF 추천 함수 정의 완료 (차원 문제 해결)


## 6. 개인 추천 함수 (Individual Recommendation)

In [7]:
def recommend_for_individual(user_id, model_type='sbert', top_k=10):
    """개인 사용자를 위한 추천 함수"""
    try:
        # 사용자 시청기록 조회
        history_ids = get_user_history_ids(user_id)
        
        if not history_ids:
            return {'error': f'User {user_id}의 시청기록이 없습니다.'}
        
        print(f"User {user_id}의 시청기록: {len(history_ids)}개")
        
        # 모델 타입에 따른 추천
        if model_type.lower() == 'sbert':
            recommendations = get_sbert_recommendations(history_ids, top_k)
        elif model_type.lower() == 'tfidf':
            recommendations = get_tfidf_recommendations(history_ids, top_k)
        else:
            return {'error': 'model_type은 sbert 또는 tfidf여야 합니다.'}
        
        return {
            'user_id': user_id,
            'model_type': model_type,
            'watched_count': len(history_ids),
            'recommendations': recommendations
        }
    
    except Exception as e:
        return {'error': str(e)}

print("개인 추천 함수 정의 완료")

개인 추천 함수 정의 완료


## 7. 그룹 추천 함수 (Group Recommendation)

In [8]:
def recommend_for_group(group_id, model_type='sbert', top_k=10):
    """그룹을 위한 추천 함수"""
    try:
        # 그룹 시청기록 조회
        history_ids = get_group_history_ids(group_id)
        
        if not history_ids:
            return {'error': f'Group {group_id}의 시청기록이 없습니다.'}
        
        print(f"Group {group_id}의 시청기록: {len(history_ids)}개")
        
        # 모델 타입에 따른 추천
        if model_type.lower() == 'sbert':
            recommendations = get_sbert_recommendations(history_ids, top_k)
        elif model_type.lower() == 'tfidf':
            recommendations = get_tfidf_recommendations(history_ids, top_k)
        else:
            return {'error': 'model_type은 sbert 또는 tfidf여야 합니다.'}
        
        return {
            'group_id': group_id,
            'model_type': model_type,
            'watched_count': len(history_ids),
            'recommendations': recommendations
        }
    
    except Exception as e:
        return {'error': str(e)}

print("그룹 추천 함수 정의 완료")

그룹 추천 함수 정의 완료


## 8. 추천 성능 테스트

In [9]:
# 유효한 사용자 및 그룹 ID (api_search.ipynb 결과 기반)
valid_user_ids = [1, 2]  # 28개, 2개의 시청기록
valid_group_ids = [1, 2, 3, 4, 5, 6, 7]  # 각각 28개의 시청기록

print("테스트할 유효한 ID:")
print(f"User IDs: {valid_user_ids}")
print(f"Group IDs: {valid_group_ids}")

# 성능 테스트 함수
def test_recommendation_performance():
    results = []
    
    # 사용자 추천 테스트
    for user_id in valid_user_ids[:2]:  # 처음 2명만 테스트
        print(f"\n=== User {user_id} 추천 테스트 ===")
        
        # SBERT 추천
        start_time = time.time()
        sbert_result = recommend_for_individual(user_id, 'sbert', 5)
        sbert_time = time.time() - start_time
        
        # TF-IDF 추천
        start_time = time.time()
        tfidf_result = recommend_for_individual(user_id, 'tfidf', 5)
        tfidf_time = time.time() - start_time
        
        if 'error' not in sbert_result and 'error' not in tfidf_result:
            results.append({
                'type': 'user',
                'id': user_id,
                'sbert_time': sbert_time,
                'tfidf_time': tfidf_time,
                'sbert_avg_similarity': np.mean([r['similarity'] for r in sbert_result['recommendations']]),
                'tfidf_avg_similarity': np.mean([r['similarity'] for r in tfidf_result['recommendations']])
            })
            
            print(f"SBERT: {sbert_time:.4f}초, 평균 유사도: {results[-1]['sbert_avg_similarity']:.3f}")
            print(f"TF-IDF: {tfidf_time:.4f}초, 평균 유사도: {results[-1]['tfidf_avg_similarity']:.3f}")
    
    # 그룹 추천 테스트
    for group_id in valid_group_ids[:2]:  # 처음 2개 그룹만 테스트
        print(f"\n=== Group {group_id} 추천 테스트 ===")
        
        # SBERT 추천
        start_time = time.time()
        sbert_result = recommend_for_group(group_id, 'sbert', 5)
        sbert_time = time.time() - start_time
        
        # TF-IDF 추천
        start_time = time.time()
        tfidf_result = recommend_for_group(group_id, 'tfidf', 5)
        tfidf_time = time.time() - start_time
        
        if 'error' not in sbert_result and 'error' not in tfidf_result:
            results.append({
                'type': 'group',
                'id': group_id,
                'sbert_time': sbert_time,
                'tfidf_time': tfidf_time,
                'sbert_avg_similarity': np.mean([r['similarity'] for r in sbert_result['recommendations']]),
                'tfidf_avg_similarity': np.mean([r['similarity'] for r in tfidf_result['recommendations']])
            })
            
            print(f"SBERT: {sbert_time:.4f}초, 평균 유사도: {results[-1]['sbert_avg_similarity']:.3f}")
            print(f"TF-IDF: {tfidf_time:.4f}초, 평균 유사도: {results[-1]['tfidf_avg_similarity']:.3f}")
    
    return results

print("\n성능 테스트 함수 정의 완료")

테스트할 유효한 ID:
User IDs: [1, 2]
Group IDs: [1, 2, 3, 4, 5, 6, 7]

성능 테스트 함수 정의 완료


## 9. 실제 추천 시스템 테스트 실행

In [10]:
# 성능 테스트 실행
print("=== 추천 시스템 성능 테스트 시작 ===")
test_results = test_recommendation_performance()

print("\n=== 성능 테스트 결과 요약 ===")
if test_results:
    sbert_times = [r['sbert_time'] for r in test_results]
    tfidf_times = [r['tfidf_time'] for r in test_results]
    sbert_similarities = [r['sbert_avg_similarity'] for r in test_results]
    tfidf_similarities = [r['tfidf_avg_similarity'] for r in test_results]
    
    print(f"평균 응답 시간:")
    print(f"  SBERT: {np.mean(sbert_times):.4f}초")
    print(f"  TF-IDF: {np.mean(tfidf_times):.4f}초")
    
    print(f"\n평균 유사도:")
    print(f"  SBERT: {np.mean(sbert_similarities):.3f}")
    print(f"  TF-IDF: {np.mean(tfidf_similarities):.3f}")
    
    print(f"\n속도 비교: TF-IDF가 SBERT보다 {np.mean(sbert_times)/np.mean(tfidf_times):.1f}배 빠름")
    print(f"유사도 비교: SBERT가 TF-IDF보다 {((np.mean(sbert_similarities) - np.mean(tfidf_similarities))/np.mean(tfidf_similarities)*100):+.1f}% 높음")
else:
    print("테스트 결과가 없습니다.")

=== 추천 시스템 성능 테스트 시작 ===

=== User 1 추천 테스트 ===
User 1의 시청기록: 28개
User 1의 시청기록: 28개
SBERT: 0.0430초, 평균 유사도: 0.860
TF-IDF: 0.4232초, 평균 유사도: 0.292

=== User 2 추천 테스트 ===
User 2의 시청기록: 12개
User 2의 시청기록: 12개
SBERT: 0.0351초, 평균 유사도: 0.863
TF-IDF: 0.4290초, 평균 유사도: 0.409

=== Group 1 추천 테스트 ===
Group 1의 시청기록: 28개
Group 1의 시청기록: 28개
SBERT: 0.0310초, 평균 유사도: 0.860
TF-IDF: 0.4052초, 평균 유사도: 0.292

=== Group 2 추천 테스트 ===
Group 2의 시청기록: 36개
Group 2의 시청기록: 36개
SBERT: 0.0316초, 평균 유사도: 0.860
TF-IDF: 0.4044초, 평균 유사도: 0.356

=== 성능 테스트 결과 요약 ===
평균 응답 시간:
  SBERT: 0.0352초
  TF-IDF: 0.4155초

평균 유사도:
  SBERT: 0.861
  TF-IDF: 0.337

속도 비교: TF-IDF가 SBERT보다 0.1배 빠름
유사도 비교: SBERT가 TF-IDF보다 +155.2% 높음


## 10. 샘플 추천 결과 확인

In [16]:
history_ids = get_user_history_ids(1)

watched_mask = np.isin(df['id'].values, history_ids)
df[watched_mask]['title'].values

array(['Sankofa', 'The Starling', 'Jeans', 'Grown Ups', 'Dark Skies',
       'Paranoia', 'Birth of the Dragon', 'Jaws', 'Jaws 2',
       'InuYasha the Movie: Affections Touching Across Time',
       'Naruto Shippûden the Movie: The Will of Fire',
       'Naruto the Movie: Ninja Clash in the Land of Snow',
       'Omo Ghetto: the Saga', 'Dhanak', 'In the Cut', 'Dear John',
       'Do the Right Thing', 'El patrón, radiografía de un crimen',
       'Extraction', 'Freedom Writers', 'Green Lantern', 'Inception',
       'The Strangers: Prey at Night', 'Get the Grift',
       'August: Osage County', 'The Pianist', 'Lawless',
       'The End of Evangelion'], dtype=object)

In [11]:
# User 1에 대한 상세 추천 결과 확인
print("=== User 1 상세 추천 결과 ===")

# SBERT 추천
print("\n[SBERT 추천 결과]")
sbert_user1 = recommend_for_individual(1, 'sbert', 5)
if 'error' not in sbert_user1:
    print(f"시청기록: {sbert_user1['watched_count']}개")
    for i, rec in enumerate(sbert_user1['recommendations'], 1):
        print(f"{i}. {rec['title']} (유사도: {rec['similarity']:.3f})")
        print(f"   장르: {rec['genre']}, 타입: {rec['type']}")
else:
    print(f"오류: {sbert_user1['error']}")

# TF-IDF 추천
print("\n[TF-IDF 추천 결과]")
tfidf_user1 = recommend_for_individual(1, 'tfidf', 5)
if 'error' not in tfidf_user1:
    print(f"시청기록: {tfidf_user1['watched_count']}개")
    for i, rec in enumerate(tfidf_user1['recommendations'], 1):
        print(f"{i}. {rec['title']} (유사도: {rec['similarity']:.3f})")
        print(f"   장르: {rec['genre']}, 타입: {rec['type']}")
else:
    print(f"오류: {tfidf_user1['error']}")

=== User 1 상세 추천 결과 ===

[SBERT 추천 결과]
User 1의 시청기록: 28개
시청기록: 28개
1. IBOY (유사도: 0.867)
   장르: International Movies, Sci-Fi & Fantasy, Thrillers, 타입: Movie
2. The Lovebirds (유사도: 0.867)
   장르: Action & Adventure, Comedies, Romantic Movies, 타입: Movie
3. Earth and Blood (유사도: 0.857)
   장르: Dramas, International Movies, Thrillers, 타입: Movie
4. The Lovely Bones (유사도: 0.856)
   장르: Dramas, International Movies, Sci-Fi & Fantasy, 타입: Movie
5. Sleight (유사도: 0.852)
   장르: Dramas, Independent Movies, Sci-Fi & Fantasy, 타입: Movie

[TF-IDF 추천 결과]
User 1의 시청기록: 28개
시청기록: 28개
1. EVANGELION: DEATH (TRUE)² (유사도: 0.293)
   장르: Action & Adventure, Anime Features, International Movies, 타입: Movie
2. Naruto Shippûden the Movie: Bonds (유사도: 0.293)
   장르: Action & Adventure, Anime Features, International Movies, 타입: Movie
3. Naruto Shippuden: The Movie: The Lost Tower (유사도: 0.291)
   장르: Action & Adventure, Anime Features, International Movies, 타입: Movie
4. Naruto Shippuden: The Movie (유사도: 0.290)
   장르: Act

## 11. FastAPI용 함수 최종 버전

In [12]:
def recommendForIndividual_sbert(user_id: int, top_k: int = 9):
    """FastAPI용 SBERT 개인 추천 함수"""
    try:
        history_ids = get_user_history_ids(user_id)
        if not history_ids:
            raise Exception("유저 시청 기록에 해당하는 콘텐츠가 없습니다.")
        
        recommendations = get_sbert_recommendations(history_ids, top_k)
        
        # FastAPI 반환 형식에 맞게 변환
        result = []
        for rec in recommendations:
            result.append({
                'id': rec['id'],
                'title': rec['title'],
                'director': rec['director'],
                'rating': rec['rating'],
                'type': rec['type']
            })
        
        return result
    
    except Exception as e:
        raise Exception(str(e))

def recommendForGroup_sbert(group_id: int, top_k: int = 9):
    """FastAPI용 SBERT 그룹 추천 함수"""
    try:
        history_ids = get_group_history_ids(group_id)
        if not history_ids:
            raise Exception("그룹 시청 기록에 해당하는 콘텐츠가 없습니다.")
        
        recommendations = get_sbert_recommendations(history_ids, top_k)
        
        # FastAPI 반환 형식에 맞게 변환
        result = []
        for rec in recommendations:
            result.append({
                'id': rec['id'],
                'title': rec['title'],
                'director': rec['director'],
                'rating': rec['rating'],
                'type': rec['type']
            })
        
        return result
    
    except Exception as e:
        raise Exception(str(e))

print("FastAPI용 추천 함수 정의 완료")
print("\n다음과 같이 app.py에서 사용 가능:")
print("result = recommendForIndividual_sbert(user_id)")
print("result = recommendForGroup_sbert(group_id)")

FastAPI용 추천 함수 정의 완료

다음과 같이 app.py에서 사용 가능:
result = recommendForIndividual_sbert(user_id)
result = recommendForGroup_sbert(group_id)
